In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import numpy as np
import re

In [2]:
df=pd.read_csv('dataset/table_2_f99.csv') # 載入資料

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 31 columns):
user                    int64
time_stamp              int64
adgroup_id              int64
pid                     object
clk                     int64
cate_id                 int64
campaign_id             int64
customer                int64
price                   float64
cms_segid               int64
cms_group_id            int64
final_gender_code       int64
age_level               int64
pvalue_level            int64
shopping_level          int64
occupation              int64
new_user_class_level    int64
user_cate               object
pv_1                    int64
cart_1                  int64
fav_1                   int64
buy_1                   int64
pv_2                    int64
cart_2                  int64
fav_2                   int64
buy_2                   int64
pv_3                    int64
cart_3                  int64
fav_3                   int64
buy_3      

In [4]:
df.columns

Index(['user', 'time_stamp', 'adgroup_id', 'pid', 'clk', 'cate_id',
       'campaign_id', 'customer', 'price', 'cms_segid', 'cms_group_id',
       'final_gender_code', 'age_level', 'pvalue_level', 'shopping_level',
       'occupation', 'new_user_class_level', 'user_cate', 'pv_1', 'cart_1',
       'fav_1', 'buy_1', 'pv_2', 'cart_2', 'fav_2', 'buy_2', 'pv_3', 'cart_3',
       'fav_3', 'buy_3', 'brand_New'],
      dtype='object')

In [5]:
to_Obj =['user', 'adgroup_id', 'pid', 'clk', 'cate_id',
       'campaign_id', 'customer', 'cms_segid', 'cms_group_id',
       'final_gender_code', 'age_level', 'pvalue_level', 'shopping_level',
       'occupation', 'new_user_class_level', 'user_cate','brand_New']
for col in to_Obj:
    df[col]=df[col].astype(str)


In [6]:
to_int8 =['pv_1', 'cart_1',
       'fav_1', 'buy_1', 'pv_2', 'cart_2', 'fav_2', 'buy_2', 'pv_3', 'cart_3',
       'fav_3', 'buy_3']
for col in to_int8:
    df[col]=df[col].astype(np.int8)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 31 columns):
user                    object
time_stamp              int64
adgroup_id              object
pid                     object
clk                     object
cate_id                 object
campaign_id             object
customer                object
price                   float64
cms_segid               object
cms_group_id            object
final_gender_code       object
age_level               object
pvalue_level            object
shopping_level          object
occupation              object
new_user_class_level    object
user_cate               object
pv_1                    int8
cart_1                  int8
fav_1                   int8
buy_1                   int8
pv_2                    int8
cart_2                  int8
fav_2                   int8
buy_2                   int8
pv_3                    int8
cart_3                  int8
fav_3                   int8
buy_3   

In [43]:
 df['user']=df['user'].astype(int)

In [44]:
df_10k = df[df.loc[:,'user']<109]
df_10k.shape

(3192, 31)

In [45]:
 df['user']=df['user'].astype(str)

In [46]:
# df_10k = df.head(2000000)   ##切一小部分data出來試

In [47]:
def trainTestValSplit(df):
    # 將資料切分為訓練(70%),測試(15%)和驗證(15%)三份。
    train=df.sample(frac=0.7)
    test=df.drop( train.index )
    val=test.sample(frac=0.5)
    test=test.drop( val.index)
    return train,test,val

def dfXYSplit(df,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetName)
    dfY=df[targetName]
    
    return dfX,dfY

In [48]:
# train,test,val=trainTestValSplit(df)    #將整份data分成 train set, test set, validation set 三份

# trainX,trainY=dfXYSplit(train,"clk")    #將training set 分成trainX & trainY
# testX,testY=dfXYSplit(test,"clk")       #將test set 分成trainX & trainY
# valX,valY=dfXYSplit(val,"clk")          #將validation set 分成trainX & trainY

In [49]:
train,test,val=trainTestValSplit(df_10k)    #將整份data分成 train set, test set, validation set 三份

trainX,trainY=dfXYSplit(train,"clk")    #將training set 分成trainX & trainY
testX,testY=dfXYSplit(test,"clk")       #將test set 分成trainX & trainY
valX,valY=dfXYSplit(val,"clk")          #將validation set 分成trainX & trainY

### Decision Tree

In [50]:
from sklearn import tree
from sklearn.metrics import classification_report

In [51]:
%%time
clf=tree.DecisionTreeClassifier(class_weight="balanced",criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
model=clf.fit(trainX,trainY)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22 ms


In [52]:
%%time
testPredY=model.predict(testX)
valPredY=model.predict(valX)

print( classification_report(testY,testPredY) )
print( classification_report(valY,valPredY) )

             precision    recall  f1-score   support

          0       0.98      0.62      0.76       459
          1       0.07      0.70      0.14        20

avg / total       0.94      0.63      0.74       479

             precision    recall  f1-score   support

          0       0.97      0.66      0.79       447
          1       0.14      0.75      0.23        32

avg / total       0.92      0.67      0.75       479

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 14.3 ms


In [53]:
tree.DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Random Forest

In [54]:
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.ensemble.forest import RandomForestClassifier

In [55]:
trainXNew = np.concatenate([trainX,valX],axis=0)
trainYNew = np.concatenate([trainY,valY],axis=0)

In [56]:
assert valX.shape[0]+trainX.shape[0] == trainXNew.shape[0]
assert valY.shape[0]+trainY.shape[0] == trainYNew.shape[0]

max_depth=5

In [57]:
clf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)
model=clf.fit(trainX,trainY)

In [58]:
model

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [59]:
model.oob_score_

0.8003581020590869

In [60]:
%%time
testPredY=model.predict(testX)
valPredY=model.predict(valX)

print( classification_report(testY,testPredY) )
print( classification_report(valY,valPredY) )

             precision    recall  f1-score   support

          0       0.96      0.82      0.89       459
          1       0.06      0.25      0.09        20

avg / total       0.92      0.80      0.85       479

             precision    recall  f1-score   support

          0       0.95      0.83      0.88       447
          1       0.12      0.34      0.18        32

avg / total       0.89      0.80      0.84       479

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 30.9 ms


n_estimators

In [61]:
clf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)
model=clf.fit(trainX,trainY)

In [62]:
model

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [63]:
model.oob_score_

0.7788719785138765

In [64]:
%%time
testPredY=model.predict(testX)
valPredY=model.predict(valX)

print( classification_report(testY,testPredY) )
print( classification_report(valY,valPredY) )

             precision    recall  f1-score   support

          0       0.97      0.81      0.88       459
          1       0.09      0.45      0.16        20

avg / total       0.93      0.80      0.85       479

             precision    recall  f1-score   support

          0       0.96      0.82      0.88       447
          1       0.17      0.53      0.26        32

avg / total       0.91      0.80      0.84       479

CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 56.5 ms


In [74]:
testX.columns

Index(['user', 'time_stamp', 'adgroup_id', 'pid', 'cate_id', 'campaign_id',
       'customer', 'price', 'cms_segid', 'cms_group_id', 'final_gender_code',
       'age_level', 'pvalue_level', 'shopping_level', 'occupation',
       'new_user_class_level', 'user_cate', 'pv_1', 'cart_1', 'fav_1', 'buy_1',
       'pv_2', 'cart_2', 'fav_2', 'buy_2', 'pv_3', 'cart_3', 'fav_3', 'buy_3',
       'brand_New'],
      dtype='object')

In [65]:
model.feature_importances_

array([0.0888976 , 0.06651828, 0.05622548, 0.00722002, 0.04391669,
       0.04718181, 0.05243038, 0.0504163 , 0.02887262, 0.03027889,
       0.00674606, 0.01922285, 0.02968942, 0.00911726, 0.01419357,
       0.11236504, 0.07722387, 0.0298637 , 0.00458955, 0.00199579,
       0.        , 0.05221298, 0.0130674 , 0.00639422, 0.        ,
       0.09037976, 0.01709153, 0.00941888, 0.00331887, 0.03115117])

gridsearch

In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
clf = RandomForestClassifier(n_jobs=-1)

In [68]:
param_grid = { 
    'n_estimators': [100, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy'],
    
}

In [69]:
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5)
CV_rfc.fit(trainX, trainY)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 300], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [70]:
model=clf.fit(trainX,trainY)

testPredY=model.predict(testX)
valPredY=model.predict(valX)

print( classification_report(testY,testPredY) )
print( classification_report(valY,valPredY) )

             precision    recall  f1-score   support

          0       0.96      0.99      0.98       459
          1       0.25      0.05      0.08        20

avg / total       0.93      0.95      0.94       479

             precision    recall  f1-score   support

          0       0.93      1.00      0.96       447
          1       0.00      0.00      0.00        32

avg / total       0.87      0.93      0.90       479



In [71]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 100}

In [72]:
rfc1=RandomForestClassifier(max_features='auto', n_estimators= 100, max_depth=4, criterion='gini')

In [73]:
model=clf.fit(trainX,trainY)

testPredY=model.predict(testX)
valPredY=model.predict(valX)

print( classification_report(testY,testPredY) )
print( classification_report(valY,valPredY) )

             precision    recall  f1-score   support

          0       0.96      0.98      0.97       459
          1       0.00      0.00      0.00        20

avg / total       0.92      0.94      0.93       479

             precision    recall  f1-score   support

          0       0.93      1.00      0.96       447
          1       0.00      0.00      0.00        32

avg / total       0.87      0.93      0.90       479

